In [1]:
import pandas as pd
import numpy as np
import json
import logic
from io import StringIO

## Load Data and describe data frame

In [2]:
applications_df = pd.read_csv('data.csv')
applications_df.application_date = pd.to_datetime(applications_df.application_date, format='mixed')

In [3]:
applications_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                1000 non-null   float64            
 1   application_date  1000 non-null   datetime64[ns, UTC]
 2   contracts         495 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 23.6+ KB


In [4]:
applications_df.head(10)

,id,application_date,contracts
0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212.0,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214.0,2024-02-12 19:24:56.857000+00:00,NaN
5,2925215.0,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
6,2925216.0,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
7,2925217.0,2024-02-12 19:39:43.431000+00:00,NaN
8,2925218.0,2024-02-12 19:40:15.507000+00:00,"[{""contract_id"": """", ""bank"": ""062"", ""summa"": ""..."
9,2925219.0,2024-02-12 19:40:21.729000+00:00,NaN


### Show example of json contracts data

In [5]:
json.loads(applications_df.contracts[1])[0]

{'contract_id': 522530,
 'bank': '003',
 'summa': 500000000,
 'loan_summa': 0,
 'claim_date': '13.02.2020',
 'claim_id': 609965,
 'contract_date': '17.02.2020'}

## Process data and generate new features

Set deafult values for features

In [6]:
applications_df['tot_claim_cnt_l180d'] = -3
applications_df['disb_bank_loan_wo_tbc'] = -3
applications_df['day_sinlastloan'] = -3
applications_df['tot_active_loans_cnt'] = -3
applications_df['tot_active_loans_sum'] = -3
applications_df['avg_loan_sum'] = -3
applications_df['avg_days_between_contracts'] = -3

Helpers and apply functions

In [7]:
def parse_contracts(contracts) -> pd.DataFrame:
    contracts_typed = json.loads(contracts)
    if type(contracts_typed) is list: # if json is list of object
        try:
            contracts_df = pd.DataFrame(data=contracts_typed)
        except ValueError as e:
            print(contracts)
            raise e
    else:  # if only one object
        contracts_df = pd.DataFrame([contracts_typed])
    if 'bank' not in contracts_df.columns:
        contracts_df['bank'] = np.nan
    contracts_df.loan_summa = contracts_df.loan_summa.apply(lambda x: x if x != '' else np.nan)
    contracts_df.contract_date = contracts_df.contract_date.apply(lambda x: x if x != '' else np.nan)
    contracts_df.contract_date = pd.to_datetime(contracts_df.contract_date, format='%d.%m.%Y')
    contracts_df.claim_date = pd.to_datetime(contracts_df.claim_date, format='%d.%m.%Y')
    return contracts_df

In [8]:
def apply_features(application):
    if pd.isnull(application.contracts):
        return application
    contracts_df = parse_contracts(application.contracts)
    logic.apply_tot_claim_cnt_l180d(application, contracts_df)
    logic.apply_disb_bank_loan_wo_tbc(application, contracts_df)
    logic.apply_day_sinlastloan(application, contracts_df)
    logic.apply_tot_active_loans(application, contracts_df)
    logic.apply_avg_days_between_contracts(application, contracts_df)
    return application

In [9]:
applications_df = applications_df.apply(apply_features, axis=1)
features_columns = [
    'tot_claim_cnt_l180d', 
    'disb_bank_loan_wo_tbc', 
    'day_sinlastloan', 
    'tot_active_loans_cnt', 
    'tot_active_loans_sum', 
    'avg_loan_sum', 
    'avg_days_between_contracts'
]
features_df = applications_df[features_columns]

In [10]:
features_df.head(10)

,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan,tot_active_loans_cnt,tot_active_loans_sum,avg_loan_sum,avg_days_between_contracts
0,-3,-3,-3,-3,-3,-3,-3.0
1,22,0,427,0,-1,-1,343.0
2,-3,-3,-3,-3,-3,-3,-3.0
3,22,0,427,0,-1,-1,343.0
4,-3,-3,-3,-3,-3,-3,-3.0
5,22,0,427,0,-1,-1,343.0
6,22,0,427,0,-1,-1,343.0
7,-3,-3,-3,-3,-3,-3,-3.0
8,13,0,288,1,99995700,99995700,0.0
9,-3,-3,-3,-3,-3,-3,-3.0


In [11]:
features_df.to_csv('contract_features.csv', index=False)

### Example how to use tot_active_loans_cnt and tot_active_loans_sum features

In [12]:
features_df[applications_df.disb_bank_loan_wo_tbc != applications_df.tot_active_loans_sum]

,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan,tot_active_loans_cnt,tot_active_loans_sum,avg_loan_sum,avg_days_between_contracts
1,22,0,427,0,-1,-1,343.0
3,22,0,427,0,-1,-1,343.0
5,22,0,427,0,-1,-1,343.0
6,22,0,427,0,-1,-1,343.0
8,13,0,288,1,99995700,99995700,0.0
...,...,...,...,...,...,...,...
975,0,0,1359,0,-1,-1,153.5
979,2,0,-1,0,-1,-1,-1.0
981,2,0,-1,0,-1,-1,-1.0
985,2,0,-1,0,-1,-1,-1.0
